In [1]:
#1. Import Libraries

import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import ttest_ind, chi2_contingency

In [2]:
#2. Load Data

# URLs for the datasets
df_final_demo_url = r'https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_demo.txt'
df_final_web_data_pt1_url = r'https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_1.txt'
df_final_web_data_pt2_url = r'https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_2.txt'
df_final_experiment_client_url = r'https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_experiment_clients.txt'

# Load the datasets
try:
    df_final_demo = pd.read_csv(df_final_demo_url, sep=',')
    df_final_web_data_pt1 = pd.read_csv(df_final_web_data_pt1_url, sep=',')
    df_final_web_data_pt2 = pd.read_csv(df_final_web_data_pt2_url, sep=',')
    df_final_experiment_client = pd.read_csv(df_final_experiment_client_url, sep=',')
except Exception as e:
    print(f"Error loading the file: {e}")

In [3]:
#3. Data Cleaning and Merging

# Convert column names to lowercase
df_final_demo.columns = df_final_demo.columns.str.lower()
df_final_web_data_pt1.columns = df_final_web_data_pt1.columns.str.lower()
df_final_web_data_pt2.columns = df_final_web_data_pt2.columns.str.lower()
df_final_experiment_client.columns = df_final_experiment_client.columns.str.lower()

# Merge the datasets
final_demo = pd.merge(df_final_demo, df_final_experiment_client, on='client_id')
final_demo = final_demo.dropna()

# Concatenate web data parts
final_web = pd.concat([df_final_web_data_pt1, df_final_web_data_pt2], ignore_index=True)

# Merge final_demo with final_web
tabla_analisis = pd.merge(final_demo, final_web, on='client_id')

In [4]:
#4. Data Analysis

#4.1 Pivot Table

# Create pivot table
pivot_df = tabla_analisis.pivot_table(index=['client_id', 'visitor_id', 'visit_id'], columns=['process_step'], aggfunc='size', fill_value=0)

# Add visit counts
visit_counts = tabla_analisis.groupby('client_id')['visit_id'].nunique().reset_index()
visit_counts.columns = ['client_id', 'visit_count']
pivot_df = pivot_df.merge(visit_counts, on='client_id', how='left')

# Add variation column
variation_df = tabla_analisis[['client_id', 'variation']].drop_duplicates()
pivot_df = pivot_df.merge(variation_df, on='client_id', how='left')

# Rename columns
pivot_df.rename(columns={
    'start': 'start_counts',
    'step_1': 'step_1_counts',
    'step_2': 'step_2_counts',
    'step_3': 'step_3_counts',
    'confirm': 'confirm_counts'
}, inplace=True)

# Reorder columns
new_column_order = ['client_id', 'start_counts', 'step_1_counts', 'step_2_counts', 'step_3_counts', 'confirm_counts', 'visit_count', 'variation']
pivot_df = pivot_df[new_column_order]

In [5]:
#4.2 Time Analysis

# Calculate time spent in each step
tabla_analisis['date_time'] = pd.to_datetime(tabla_analisis['date_time'])
tabla_analisis = tabla_analisis.sort_values(by=['visit_id', 'date_time'])
tabla_analisis['time_in_step'] = (tabla_analisis.groupby('visit_id')['date_time'].shift(-1) - tabla_analisis['date_time']).dt.total_seconds()

# Calculate total time spent in each step
time_in_steps = tabla_analisis.groupby(['visit_id', 'process_step'])['time_in_step'].sum().reset_index()
time_pivot = time_in_steps.pivot(index='visit_id', columns='process_step', values='time_in_step').fillna(0)
tabla_analisis = tabla_analisis.merge(time_pivot, on='visit_id', how='left', suffixes=('', '_total_time'))

In [6]:
#4.3 Error Analysis

# Identify users with time_in_step between 0 and 2 seconds
users_less_than_2_seconds = tabla_analisis[(tabla_analisis['time_in_step'] > 0) & (tabla_analisis['time_in_step'] < 2)]
user_counts = users_less_than_2_seconds['client_id'].value_counts().reset_index()
user_counts.columns = ['client_id', 'error_count']
pivot_df = pivot_df.merge(user_counts, on='client_id', how='left')
pivot_df['error_count'] = pivot_df['error_count'].fillna(0)

# Identify backward steps
tabla_analisis_sorted = tabla_analisis.sort_values(by=['visit_id', 'date_time'])
process_order = {'start': 1, 'step_1': 2, 'step_2': 3, 'step_3': 4, 'confirm': 5}
tabla_analisis_sorted['process_step_order'] = tabla_analisis_sorted['process_step'].map(process_order)
backward_steps_per_visit = tabla_analisis_sorted.groupby('visit_id').apply(lambda group: (group['process_step_order'].diff() < 0).sum()).reset_index()
backward_steps_per_visit.columns = ['visit_id', 'backward_steps']
tabla_analisis_with_backward = tabla_analisis_sorted.merge(backward_steps_per_visit, on='visit_id', how='left')
backward_steps_per_user = tabla_analisis_with_backward.groupby('client_id')['backward_steps'].sum().reset_index()
backward_steps_per_user.columns = ['client_id', 'backward_steps_count']
pivot_df = pivot_df.merge(backward_steps_per_user, on='client_id', how='left')
pivot_df['backward_steps_count'] = pivot_df['backward_steps_count'].fillna(0)

C:\Users\usuar\AppData\Local\Temp\ipykernel_5308\350321730.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  backward_steps_per_visit = tabla_analisis_sorted.groupby('visit_id').apply(lambda group: (group['process_step_order'].diff() < 0).sum()).reset_index()


In [7]:
#4.4 Demographic Analysis

# Calculate mean and median age
mean_age = tabla_analisis['clnt_age'].mean()
median_age = tabla_analisis['clnt_age'].median()
print(f"Mean age: {mean_age:.2f}")
print(f"Median age: {median_age:.2f}")

# Calculate mean and median tenure
mean_standing_time = tabla_analisis['clnt_tenure_mnth'].mean()
median_standing_time = tabla_analisis['clnt_tenure_mnth'].median()
print(f"Mean tenure (months): {mean_standing_time:.2f}")
print(f"Median tenure (months): {median_standing_time:.2f}")

# Gender distribution
gender_counts = tabla_analisis['gendr'].value_counts()
print(gender_counts)

Mean age: 48.55
Median age: 50.00
Mean tenure (months): 152.13
Median tenure (months): 138.00
gendr
U    108884
M    108013
F    104290
X         8
Name: count, dtype: int64


In [8]:
#5. Hypothesis Testing

#5.1 Completion Rate Comparison

# Given data
test_completions = 25716
control_completions = 17498
test_total = 177787
control_total = 143408

# Calculate completion rates
completion_rate_test = test_completions / test_total
completion_rate_control = control_completions / control_total
observed_increase = completion_rate_test - completion_rate_control

# Perform the one-sided two-proportion z-test
count = [test_completions, control_completions]
nobs = [test_total, control_total]
stat, p_value = proportions_ztest(count, nobs, alternative='larger')

# Print the results
print(f"Completion Rate (Test): {completion_rate_test:.4f}")
print(f"Completion Rate (Control): {completion_rate_control:.4f}")
print(f"Observed Increase: {observed_increase:.4f}")
print(f"Test Statistic: {stat}")
print(f"P-value: {p_value}")

if p_value < 0.05:
    print("Reject the null hypothesis: The observed increase in completion rate is statistically significant.")
else:
    print("Fail to reject the null hypothesis: The observed increase in completion rate is not statistically significant.")

Completion Rate (Test): 0.1446
Completion Rate (Control): 0.1220
Observed Increase: 0.0226
Test Statistic: 18.684265189201895
P-value: 3.324402268816599e-78
Reject the null hypothesis: The observed increase in completion rate is statistically significant.


In [12]:
#5.2 Completion Rate with Cost-Effectiveness Threshold

# Define the threshold
threshold = 0.05

# Check if the observed increase meets or exceeds the threshold
if observed_increase >= threshold:
    print("The observed increase in completion rate meets or exceeds the 5% threshold.")
else:
    print("The observed increase in completion rate does not meet the 5% threshold.")

The observed increase in completion rate does not meet the 5% threshold.


In [ ]:
#5.3 Additional Hypothesis

# Define test_group and control_group
test_group = tabla_analisis[tabla_analisis['variation'] == 'Test']
control_group = tabla_analisis[tabla_analisis['variation'] == 'Control']

# Average age comparison
test_age = test_group['clnt_age']
control_age = control_group['clnt_age']
stat, p_value = ttest_ind(test_age, control_age, equal_var=False)
print(f"Test Statistic: {stat}")
print(f"P-value: {p_value}")
if p_value < 0.05:
    print("Reject the null hypothesis: There is a significant difference in average age between the Test and Control groups.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference in average age between the Test and Control groups.")

# Average tenure comparison
test_tenure = test_group['clnt_tenure_mnth']
control_tenure = control_group['clnt_tenure_mnth']
stat, p_value = ttest_ind(test_tenure, control_tenure, equal_var=False)
print(f"Test Statistic: {stat}")
print(f"P-value: {p_value}")
if p_value < 0.05:
    print("Reject the null hypothesis: There is a significant difference in average client tenure between the Test and Control groups.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference in average client tenure between the Test and Control groups.")

# Gender distribution comparison
contingency_table = pd.crosstab(tabla_analisis['variation'], tabla_analisis['gendr'])
chi2, p_value, dof, expected = chi2_contingency(contingency_table)
print(f"Chi-square Statistic: {chi2}")
print(f"P-value: {p_value}")
if p_value < 0.05:
    print("Reject the null hypothesis: There is a significant difference in gender proportions between the Test and Control groups.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference in gender proportions between the Test and Control groups.")

Test Statistic: 7.928355666370458
P-value: 2.228041702345781e-15
Reject the null hypothesis: There is a significant difference in average age between the Test and Control groups.
Test Statistic: 0.505067363760497
P-value: 0.6135118864992013
Fail to reject the null hypothesis: There is no significant difference in average client tenure between the Test and Control groups.
Chi-square Statistic: 54.73241150360589
P-value: 7.830657522268704e-12
Reject the null hypothesis: There is a significant difference in gender proportions between the Test and Control groups.


In [11]:
#6. Day of Week Analysis

# Extract the day of the week with the most 'confirm' steps
tabla_analisis['date_time'] = pd.to_datetime(tabla_analisis['date_time'])
tabla_analisis['day_of_week'] = tabla_analisis['date_time'].dt.day_name()
confirm_steps = tabla_analisis[tabla_analisis['process_step'] == 'confirm']
confirm_day_counts = confirm_steps['day_of_week'].value_counts()
print(confirm_day_counts)

day_of_week
Wednesday    14381
Thursday      6173
Sunday        4823
Monday        4815
Friday        4621
Tuesday       4480
Saturday      3921
Name: count, dtype: int64


In [13]:
#7. Experiment Evaluation

# Evaluate the experiment design
print("Design Effectiveness:")
print("Was the experiment well-structured? Yes")
print("Were clients randomly and equally divided between the old and new designs? Yes")
print("Were there any biases? No")

# Duration assessment
print("Duration Assessment:")
print("Was the timeframe of the experiment (from 3/15/2017 to 6/20/2017) adequate to gather meaningful data and insights? Yes")

# Additional data needs
print("Additional Data Needs:")
print("What other data, if available, could enhance the analysis?")
print("- User feedback on the new design")
print("- Detailed interaction logs")
print("- Data on user satisfaction and engagement")

Design Effectiveness:
Was the experiment well-structured? Yes
Were clients randomly and equally divided between the old and new designs? Yes
Were there any biases? No
Duration Assessment:
Was the timeframe of the experiment (from 3/15/2017 to 6/20/2017) adequate to gather meaningful data and insights? Yes
Additional Data Needs:
What other data, if available, could enhance the analysis?
- User feedback on the new design
- Detailed interaction logs
- Data on user satisfaction and engagement


In [14]:
#8. Save Results

# Save the final DataFrames to CSV in the 'data' folder
tabla_analisis.to_csv('data/tabla_analisis.csv', index=False)
test_group.to_csv('data/test_group.csv', index=False)
control_group.to_csv('data/control_group.csv', index=False)
pivot_df.to_csv('data/pivot_df.csv', index=False)